In [4]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import sentencepiece as spm
import gensim
from torch.utils.data import Dataset, DataLoader
from gensim.models import Word2Vec
import torch
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import pandas as pd

import gensim
import numpy as np
import random
import math

In [5]:
train_df = pd.read_json('train_data.json', lines=True)

In [6]:
train_df.head()

,text
0,"One day, a little girl named Lily found a need..."
1,"Once upon a time, there was a little car named..."
2,"One day, a little fish named Fin was swimming ..."
3,"Once upon a time, in a land full of trees, the..."
4,"Once upon a time, there was a little girl name..."


In [7]:
train_df = train_df[:1]
train_df

,text
0,"One day, a little girl named Lily found a need..."


In [8]:
def generate_inputs_and_labels(df, sp):
    inputs = []
    labels = []

    for index, row in df.iterrows():
        text = row['text']
        tokens = text.split()  # Tokenize the text by splitting on whitespace

        # Generate input by adding start of sentence token at the beginning
        input_sequence = tokens

        # Generate label by adding end of sentence token at the end
        label_sequence = tokens

        inputs.append(input_sequence)
        labels.append(label_sequence)

    # Convert input and label sequences to strings
    input_strings = [' '.join(sequence) for sequence in inputs]
    label_strings = [' '.join(sequence) for sequence in labels]

    # Tokenize input strings and add <sos> token at the beginning
    tokenized_inputs = []
    input_ids = []
    for sequence in input_strings:
        tokenized_sequence = sp.encode_as_pieces(sequence)
        tokenized_sequence = ['<sos>'] + tokenized_sequence  # Add <sos> token manually
        input_ids.append([sp.piece_to_id('<sos>')] + sp.encode_as_ids(sequence))  # Get token IDs
        tokenized_inputs.append(tokenized_sequence)

    # Tokenize label strings and add </sos> token at the end
    tokenized_labels = []
    label_ids = []
    for sequence in label_strings:
        tokenized_sequence = sp.encode_as_pieces(sequence)
        tokenized_sequence.append('</sos>')  # Add </sos> token manually at the end
        label_ids.append(sp.encode_as_ids(sequence) + [sp.piece_to_id('</sos>')])  # Get token IDs
        tokenized_labels.append(tokenized_sequence)

    # Print tokenized input and label sequences
    for i in range(len(inputs)):
        print("Input Text:", input_strings[i])
        print("Tokenized Input:", tokenized_inputs[i])
        print("Input IDs:", input_ids[i])
        print("Label Text:", label_strings[i])
        print("Tokenized Label:", tokenized_labels[i])
        print("Label IDs:", label_ids[i])

    return input_ids, label_ids


In [9]:
# Example usage
sp = spm.SentencePieceProcessor()
sp.load('small_m.model')

source_ids, target_ids = generate_inputs_and_labels(train_df, sp)

Input Text: One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt. Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt." Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.
Tokenized Input: ['<sos>', '▁one', '▁day', ',', '▁a', '▁little', '▁girl', '▁name', 'd', '▁lily', '▁found', '▁a', '▁needle', '▁in', '▁her', '▁room', '.', '▁she', '▁knew', '▁it', '▁was', '▁difficult', '▁to', '▁play', '▁with', '▁it', '▁', 'because', '▁it', '▁was', '▁sharp', '.', '▁lily', '▁

In [10]:
# Flatten the list of lists
flat_ids = [item for sublist in source_ids for item in sublist]

# Find the maximum ID
max_id = max(flat_ids)
print("Maximum ID value in source:", max_id)

# Flatten the list of lists
flat_ids = [item for sublist in target_ids for item in sublist]

# Find the maximum ID
max_id = max(flat_ids)
print("Maximum ID value in labels:", max_id)


Maximum ID value in source: 2599
Maximum ID value in labels: 2599


# Transformer

In [11]:

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [12]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [13]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        self.d_model = d_model
        self.max_seq_length = max_seq_length
        pe = self._init_pe(max_seq_length, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_length = x.size(1)
        pe = self.pe[:seq_length, :] if seq_length <= self.max_seq_length else self._init_pe(seq_length, self.d_model)
        return x + pe

    def _init_pe(self, seq_length, d_model):
        pe = torch.zeros(seq_length, d_model)
        position = torch.arange(0, seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        return pe


In [14]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [15]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, decoder_input, decoder_mask):
        attn_output = self.self_attn(decoder_input, decoder_input, decoder_input, decoder_mask)
        x = self.norm1(decoder_input + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [16]:
class Transformer(nn.Module):
    def __init__(self, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, tgt):
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return tgt_mask

    def forward(self, decoder_input):
        decoder_mask = self.generate_mask(decoder_input)
        decoder_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(decoder_input)))

        dec_output = decoder_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, decoder_mask)

        output = self.fc(dec_output)
        return output


In [17]:
# Convert lists to tensors
source_data = torch.tensor(source_ids)
target_data = torch.tensor(target_ids)

# Print shapes
print("Source data shape:", source_data.shape)
print("Target data shape:", target_data.shape)

print(source_data)
print(target_data)

Source data shape: torch.Size([1, 165])
Target data shape: torch.Size([1, 165])
tensor([[   0,   38,   28,    6,    8,   37,   53,   86,   34,   31,  119,    8,
         1614,   21,   14,  198,    3,   11,  185,   12,    9, 1455,    7,   54,
           24,   12,   19,  230,   12,    9, 1316,    3,   31,   59,    7,  259,
            4, 1614,   24,   14,   43,    6,   23,   11,   94, 2599,    8, 1293,
           32,   14,  802,    3,   31,   68,    7,   14,   43,    5,   18,    6,
           16,  749,    6,   49,  119,  149, 1614,    3,   66,   25,  259,   12,
           24,  145,    5, 2599,  140,  802,   82,   14,   43,   76,    5,   18,
            6,   16,  257,    6,   31,    6,   96,   66,  259,    4, 1614,    5,
          524,  129,  802,   46,  104,    6,   13,  678,    4, 1614,    5, 2599,
           20,    4, 1293,   32,   31,   17,   15,  802,    3,   12,    9,   60,
         1455,   36,   64,   19,  230,   13,   50, 1901,    5,  820,  183,  125,
            3,  167,   13,  5

In [18]:
tgt_vocab_size = 5000
d_model = 64
num_heads = 4
num_layers = 3
d_ff = 128
max_seq_length = 100
dropout = 0.1

# Initialize the model
transformer = Transformer( tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)


criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(1000):
    optimizer.zero_grad()
    output = transformer(source_data)
    
    # Adjust target_data to have the same length as model output
    target_data_adjusted = target_data[:, :output.size(1)]
    
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), target_data_adjusted.contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.713146209716797
Epoch: 2, Loss: 8.490992546081543
Epoch: 3, Loss: 8.327680587768555
Epoch: 4, Loss: 8.103431701660156
Epoch: 5, Loss: 7.946131229400635
Epoch: 6, Loss: 7.802438259124756
Epoch: 7, Loss: 7.6724348068237305
Epoch: 8, Loss: 7.544612407684326
Epoch: 9, Loss: 7.40691614151001
Epoch: 10, Loss: 7.304545879364014
Epoch: 11, Loss: 7.183355331420898
Epoch: 12, Loss: 7.0761542320251465
Epoch: 13, Loss: 6.965683460235596
Epoch: 14, Loss: 6.849248886108398
Epoch: 15, Loss: 6.7380146980285645
Epoch: 16, Loss: 6.609580993652344
Epoch: 17, Loss: 6.499388217926025
Epoch: 18, Loss: 6.3643479347229
Epoch: 19, Loss: 6.234854698181152
Epoch: 20, Loss: 6.099053382873535
Epoch: 21, Loss: 5.992619037628174
Epoch: 22, Loss: 5.836920261383057
Epoch: 23, Loss: 5.724179744720459
Epoch: 24, Loss: 5.594740867614746
Epoch: 25, Loss: 5.4804792404174805
Epoch: 26, Loss: 5.3604960441589355
Epoch: 27, Loss: 5.252935409545898
Epoch: 28, Loss: 5.144591808319092
Epoch: 29, Loss: 5.01197147

In [19]:
def word_to_token_id(word, sp_model):
    # Convert word to token ID using SentencePiece
    return sp_model.piece_to_id(word)

def generate_text(model, sp_model, starting_word, ending_word, max_length):
    
    # Convert starting and ending words to token IDs
    starting_token_id = word_to_token_id(starting_word, sp_model)
    if starting_token_id is None:
        raise ValueError(f"Starting word '{starting_word}' not found in vocabulary.")
    
    ending_token_id = word_to_token_id(ending_word, sp_model)
    if ending_token_id is None:
        raise ValueError(f"Ending word '{ending_word}' not found in vocabulary.")
    
    
    generated_sequence = [starting_token_id]
    with torch.no_grad():
        for _ in range(max_length):
            input_tensor = torch.tensor([generated_sequence])
            output = model(input_tensor)
            print(f"output is {output}, \noutput shape is {output.shape} ")
            
            predicted_token = output.argmax(-1)[:,-1].item()
            print(f"predicted_token is {predicted_token}")
            
            generated_sequence.append(predicted_token)
            print(f"generated_sequence is {generated_sequence}")
            
            if predicted_token == ending_token_id:
                break
            print("-------")
            
    # Convert token IDs to words using SentencePiece
    generated_text = sp_model.decode_ids(generated_sequence)
    return generated_text

# Example usage:
starting_word = "<sos>"
ending_word = "</sos>"
max_length = 50
generated_sequence = generate_text(transformer, sp, starting_word, ending_word, max_length)
print("Generated sequence:", generated_sequence)

output is tensor([[[-6.0123, -4.6745, -4.3603,  ..., -4.3819, -5.9473, -5.8897]]]), 
output shape is torch.Size([1, 1, 5000]) 
predicted_token is 28
generated_sequence is [0, 28]
-------
output is tensor([[[-5.2795, -3.9530, -4.3246,  ..., -4.3539, -5.1141, -4.8554],
         [-4.8325, -4.0280, -4.1898,  ..., -4.8621, -4.9755, -4.4617]]]), 
output shape is torch.Size([1, 2, 5000]) 
predicted_token is 6
generated_sequence is [0, 28, 6]
-------
output is tensor([[[-3.2425, -3.2849, -2.8890,  ..., -3.4632, -4.0306, -3.6390],
         [-4.2873, -3.5796, -3.6699,  ..., -4.0730, -4.6746, -3.6146],
         [-3.4823, -3.0382, -4.0119,  ..., -4.2278, -4.2283, -4.1791]]]), 
output shape is torch.Size([1, 3, 5000]) 
predicted_token is 8
generated_sequence is [0, 28, 6, 8]
-------
output is tensor([[[-4.0704, -3.7313, -4.4022,  ..., -4.7641, -4.1839, -4.3341],
         [-4.5957, -4.8304, -4.8339,  ..., -4.5680, -4.9796, -5.0161],
         [-3.9045, -4.2701, -4.7003,  ..., -5.1627, -4.3758, -4.741

In [20]:
print(f"The actual trained sentence is \n{train_df['text'][0]}")

The actual trained sentence is 
One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.


In [21]:
print(f"generated_sequence for 50 words \n{generated_sequence}")

generated_sequence for 50 words 
 ⁇  day, a little girl named lily found a needle in her room. she knew it was difficult to play with it because it was sharp. lily wanted to share the needle with her mom, so she could sew a button on her shirt.


In [22]:
""""
100 epochs
⁇  day, a her mom, a button on her shirt. lily, a little girl name a button on her mom and said, 
lily wanted to share the needle and fix your shirt. lily went to her mom and sew a button on her mom
 
 
1000 epochs
⁇  day, a little girl named lily found a needle in her room. she knew it was difficult to play with 
it because it was sharp. lily wanted to share the needle with her mom, so she could sew a button on her shirt.
"""

'"\n ⁇  day, a her mom, a button on her shirt. lily, a little girl name a button on her mom and said, \n lily wanted to share the needle and fix your shirt. lily went to her mom and sew a button on her mom a'